## Factors Influencing Spread ##

**BETA RELEASE**

*This project remains in the experimental stage and there are likely many tweaks to come that will improve/change outcomes. I do believe at this stage that the simulation provides good general insights regarding the relationship between various factors and viral spread.*

**REPLICABILITY**

*Randomness is in an important factor in RKnot's approach to simulation (and frankly in real-word viral transmission), so the results of the sims below will not be repeatable with each iteration. The below examples are meant to show general differences based on state; further analysis should run the same simulation multiple times to see the mean impact.*

**SARS-COV-2**

*While sars-cov-2 is obviously topical globally, the goal of RKnot is to be applicable to any virus (or anything that spreads like a virus). sars-cov-2 is used only for illustration purposes.*

### Base US Simulation ###

To explore the various concepts of RKnot and viral spread, we'll use a simulation design based on [CDC Best Planning Scenario](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html) guidelines for
COVID-19 characteristics including:

* $R_0$ 2.5
* IFR for each of 4 age groups
    * 0-19: 0.003%
    * 20-49: 0.02%
    * 50-69: 0.5%
    * 70+: 5.4%

Other assumptions:

* Population of $10,000^1$
* Initial Infected of 2
* Duration of Infectiousness 14 days$^2$
* Duration of Immunity 365 days
* Density of ~1 subject per location (`dlevel='med'`)

$^1$*proportionately split among the 4 age groups to match US census data.*

$^2$*equal likelihood of transmission on any day (i.e. no viral load curve)*

[US Census Data](https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf)

### Natural ###

#### 1. Equal ####
The first simulation makes the most homogeneous assumptions. 

+ No group is restricted in terms of movement. 
+ All dots are able to interact with one another. 
+ All dots are susceptible at initiation.
+ All dots are equally likely to move to any dot on the grid (`mover='equal'`)

In [1]:
### HIDDEN ###
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.animation import FFMpegWriter

def modulereload(modulename):
    import imp
    imp.reload(modulename)
    
def perstr(val):
    return '{0:.0%}'.format(val)
def fatstr(val):
    return '{0:.2%}'.format(val)

from IPython.display import display, Markdown, Latex
def md(value):
    return display(Markdown(value))

def get_results(sim):
    peak = perstr(sim.log['curr_per'].max())
    i_peak = sim.log['curr_per'].argmax()
    hit = perstr(sim.log['ever_per'][i_peak])
    total = perstr(sim.log['ever_per'][sim.i_tick - 2])
    fat = fatstr(sim.log['dep_per'][sim.i_tick - 2])
    ifr = fatstr((1 / sim.log['ever_per'][sim.i_tick - 2] ) * sim.log['dep_per'][sim.i_tick - 2])
    days_to_peak = str(i_peak)
    
    group_deps = sim.log['group_dep'][sim.i_tick - 2]
    _70 = group_deps[-2:].sum() if group_deps.shape[0] == 5 else group_deps[-1]
    _70 = perstr(_70/100)
    
    return [peak, hit, total, fat, _70, ifr, days_to_peak]

def results_table(*args):
    table = 'Results: \n\n'
    table += '|      |      |'
    table += ' \n'
    table += '| ------------- |:-------------:|'
    table += ' \n'
    table += '| Peak      |   {} |'
    table += ' \n'
    table += '| HIT      | {}      |'
    table += ' \n'
    table += '| Total      | {}      |'
    table += ' \n'
    table += '| Fatalities | {}      |'
    table += ' \n'
    table += '| % > 70 | {}      |'
    table += ' \n'    
    table += '| IFR | {}      |'
    table += ' \n'    
    table += '| Days to Peak |   {} |'
    table += ' \n'

    table = table.format(*args)
    
    return table

def move_to_doc_folder():
    import os
    dir_ = os.getcwd()
    dir_docs = dir_ + '/docs/docs'
    filename = '/hit.ipynb'

    from shutil import copyfile
    copyfile(dir_ + filename, dir_docs + filename)

import shelve
file = 'vids/baseus/tables'
# db.close()
db = shelve.open(file, writeback=True)

In [2]:
### HIDDEN ###
%load_ext autoreload
%autoreload 2
IMPORT = False
RUN = False
SAVE = False
SHOW_CHARTS = False
SAVE_CHARTS = False
PATH = 'vids/baseus/tests/'

The basic layout is below. These parameters can also be imported from `rknot.sims.baseus` for convenienve.

```python
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='equal',
)
group2 = dict(
    name='20-49',
    n=4100,
    n_inf=1,
    ifr=0.0002,
    mover='equal',
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    mover='equal',
)
group4 = dict(
    name='70+',
    n=900,
    n_inf=0,
    ifr=0.054,
    mover='equal',
)
groups = [group1, group2, group3, group4]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}
```

We instantiate a new sim by passing `groups` and `params`. We can also flag `details` to get some information about the Sim structure.

```python
from rknot import Sim, Chart
sim = Sim(groups=groups, details=True, **params)
sim.run()
```

In [32]:
### HIDDEN ###
if IMPORT:
    from rknot.sims.baseus import params, groups
    from rknot import Sim, Chart

    for group in groups:
        group['mover'] = 'equal'
        
    sim = Sim(groups=groups, details=True, **params)

---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           Boundary|  [  1 100   1 100]|          Locations|             10,000|
|-------------------|-------------------|-------------------|-------------------|
|         Population|             10,000|            Density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|       Contact Rate|                1.0|                   |                   |
|-------------------|-------------------|-------------------|-------------------|


Running the animation will result in a video as per below:

```python
chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [33]:
if RUN:
    sim.run()
    print (get_results(sim))


['39%', '63%', '88%', '0.56%', '48%', '0.64%', '68']


In [38]:
 if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '1.mp4', writer=FFMpegWriter(fps=10))

In [36]:
if SAVE:
    results = get_results(sim)
    db['1'] = results

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/1.mp4' controls>alternative text</video>


*Note embedded videos are used for convenience purposes. Given the random processes involved, running the same code will produce slightly different results each time.*

In [37]:
if RUN:
    md(results_table(*db['1']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   39% | 
| HIT      | 63%      | 
| Total      | 88%      | 
| Fatalities | 0.56%      | 
| % > 70 | 48%      | 
| IFR | 0.64%      | 
| Days to Peak |   68 | 


In this scenario, **RKnot** closely replicates the curve of a standard SIR model, which expects HIT of 60% for $R_0$ of 2.5 (HIT = 1 - 1 / $R_0$).

Variation from the standard SIR model will always result given:

1. In the simulation, movement and transmission are stochastic processes.
2. This sim does not have an entirely homogeneous population, with different IFRs and varying numbers of contacts between subjects.

#### 2. Local ####
In remaining simulations, we begin to introduce ever increasing homogeneity.

Our first change is to adjust the subjects mover functions to `local`. The `local` mover has a strong bias towards locations only in its immediate vicinity, which is a better approximation of real world processes (though certainly not a perfect one).

```python
group1['mover'] = 'local'
group2['mover'] = 'local'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [44]:
### HIDDEN ###
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)

    from rknot import Sim, Chart
    assert all(g['mover'] == 'local' for g in baseus.groups)

if RUN:
    sim = Sim(groups=baseus.groups, **baseus.params)
    sim.run()
    print (get_results(sim))

['3%', '23%', '47%', '0.38%', '31%', '0.80%', '175']


In [47]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '2.mp4', writer=FFMpegWriter(fps=10))

In [45]:
if SAVE:
    results = get_results(sim)
    db['2'] = results

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/2.mp4' controls>Video Failed To Load</video>
    

In [46]:
if True:
    md(results_table(*db['2']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   3% | 
| HIT      | 23%      | 
| Total      | 47%      | 
| Fatalities | 0.38%      | 
| % > 70 | 31%      | 
| IFR | 0.80%      | 
| Days to Peak |   175 | 


Compared to [Example 1](#1.-Equal), we can see that restricting movement has a major impact on spread. The curve is flattened and extended with total infected, peak, HIT, and fatalities all reduce. 

But the virus is never completely eradicated and instead moves in a progressive wave across the grid space. Note that the infection and fatalitiy levels are somewhat artificially reduced as the virus still has a large susceptible population in the upper portion of the grid that it has not yet reached.

*CAUTION: RKnot estimates transmission rate assuming all subjects are* `equal` *movers. Thus, where* `local` *movers are used, the transmission rate may (or may not) be underestimated.*

#### 3. Social  ####
In this simulation, we set `mover=social` for just the 20-49 age group. This is a rough approximation of that group's real-world propensity to travel more frequently (or go to more events).

```python
group1['mover'] = 'local'
group2['mover'] = 'social'
group3['mover'] = 'local'
group4['mover'] = 'local'
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [5]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    
    from rknot import Sim, Chart
    
    groups = baseus.groups
    assert all(g['mover'] == 'local' for g in baseus.groups)
    
    groups[1]['mover'] = 'social'
    
if RUN:    
    sim = Sim(groups=groups, **baseus.params)
    sim.run()
    print (get_results(sim))


['34%', '58%', '84%', '0.60%', '49%', '0.71%', '65']


In [8]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '3.mp4', writer=FFMpegWriter(fps=10))

In [6]:
if SAVE:
    results = get_results(sim)
    db['3'] = results

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/3.mp4' controls>Video Failed To Load</video>

In [7]:
if RUN:
    md(results_table(*db['3']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   34% | 
| HIT      | 58%      | 
| Total      | 84%      | 
| Fatalities | 0.60%      | 
| % > 70 | 49%      | 
| IFR | 0.71%      | 
| Days to Peak |   65 | 


Comparing again to [Example 1](#1.-Equal), we can see how powerful mixing is within a population. Even with the majority of subjects moving only locally, a small group of subjects is moving more broadly across the space will significantly increase the amount of spread.

#### 4. Pre-Existing Immunity ####
In this scenario, we adjust the susceptibility factor for just two groups by relatively small amounts as follows:

+ 20-49: 80%
+ 50-69: 65%

This means, in the inverse, that 10% and 25% of the subjects in the respective groups are already immune to the virus (whether through pre-existing T-cell immunity, anti-bodies, or otherwise).

The older group is assumed to have a lower susceptibility factor as it is more likely that older people will have had more exposure to similar viruses over their lifetime.

T-cell immunity to sars-cov-2 remains a controversial subject, but [many studies](https://www.bmj.com/content/370/bmj.m3563) have found prevalance of T-cells between 20% - 50% in *people unexposed to sars-cov-2*. It is suggested that exposure to "common cold" coronaviruses (or more dangerous ones) may convey this immunity.

```python
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(sim, use_init_func=True) 
chart.animate.to_html5_video()
```

In [54]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    
    from rknot import Sim, Chart
    
    assert all(g['mover'] == 'local' for g in baseus.groups)
    
    groups = baseus.groups
    groups[1]['mover'] = 'social'
    groups[1]['susf'] = .8
    groups[2]['susf'] = .65
if RUN:
    sim = Sim(groups=groups, **baseus.params)
    sim.run()
    print (get_results(sim))


['20%', '39%', '63%', '0.41%', '38%', '0.65%', '84']


In [55]:
if SAVE:
    results = get_results(sim)
    db['4'] = results

In [57]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '4.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/4.mp4' controls>Video Failed to Load</video>
    

In [56]:
if RUN:
    md(results_table(*db['4']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   20% | 
| HIT      | 39%      | 
| Total      | 63%      | 
| Fatalities | 0.41%      | 
| % > 70 | 38%      | 
| IFR | 0.65%      | 
| Days to Peak |   84 | 


Relative to [Example 3](#3.-Social) above, pre-existing immunity would reduce all aspects of the spread curve signifcantly.

Note the reduction in fatalities results *even though the most susceptible group is assumed to NOT have pre-existing immunity*.

#### 5. Events ####
Certainly, the vast majority of people in the US do not move in such pre-defined ways as set out by the `mover` function. In reality, people tend to move with a local bias with a small number of interactions, supplemented by larger movements to locations with a large number of interactions in a small amount of time.

In RKnot, we can simulate this with [Event objects](concepts.ipynb#Events). And we will incorporate a number of them in this simulation. 

First, we will reset our group parameters by importing from `sims.baseus`.

Then we will instantiate a handful of events recurring periodically over the duration of the sim.

```python
from rknot.sims.baseus import params, groups

from rknot.events import Event

school1 = Event(
    name='school1', xy=[25,42], start_tick=2, 
    groups=[0], capacity=10, recurring=2
)
school2 = Event(
    name='school2', xy=[78,82], start_tick=3, 
    groups=[0], capacity=10, recurring=2
)
school3 = Event(
    name='school3', xy=[92,32], start_tick=4, 
    groups=[0], capacity=7, recurring=2
)
game1 = Event(
    name='game1', xy=[50,84], start_tick=6, 
    groups=[0,1,2,3], capacity=100, recurring=14
)
game2 = Event(
    name='game2', xy=[45,52], start_tick=5, 
    groups=[0,1,2], capacity=76, recurring=14
)
game3 = Event(
    name='game3', xy=[12,87], start_tick=1, 
    groups=[0,1,2], capacity=56, recurring=14
)
game4 = Event(
    name='game4', xy=[52,98], start_tick=3, 
    groups=[1,2], capacity=113, recurring=28
)
concert1 = Event(
    name='concert1', xy=[20,20], start_tick=7, 
    groups=[0,1], capacity=50, recurring=14
)                 
concert2 = Event(
    name='concert2', xy=[91,92], start_tick=28, 
    groups=[1], capacity=50, recurring=14
)
concert3 = Event(
    name='concert3', xy=[62,38], start_tick=21, 
    groups=[2,3], capacity=25, recurring=14
)
concert4 = Event(
    name='concert4', xy=[38,42], start_tick=14, 
    groups=[1,2], capacity=50, recurring=14
)
bar1 = Event(
    name='bar1', xy=[17,24], start_tick=4, 
    groups=[1], capacity=5, recurring=3
)
bar2 = Event(
    name='bar2', xy=[87,13], start_tick=5, 
    groups=[1], capacity=5, recurring=4
)
bar3 = Event(
    name='bar3', xy=[52,89], start_tick=6, 
    groups=[1,2], capacity=5, recurring=3
)
bar4 = Event(
    name='bar4', xy=[16,27], start_tick=7, 
    groups=[1,2,3], capacity=4, recurring=7
)
bar5 = Event(
    name='bar5', xy=[89,46], start_tick=6, 
    groups=[1,2], capacity=7, recurring=7
)
church = Event(
    name='church', xy=[2,91], start_tick=7, 
    groups=[2,3], capacity=20, recurring=7
)

events = [
    school1, school2, school3, game1, game2, game3, game4, 
    concert1, concert2, concert3, concert4, 
    bar1, bar2, bar3, bar4, bar5, church
]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [58]:
if IMPORT:
    from rknot import Sim, Chart
    from rknot.sims import baseus

    modulereload(baseus)    
    assert 'susf' not in baseus.groups[1]    
    assert all(g['mover'] == 'local' for g in baseus.groups)

if RUN:   
    sim = Sim(groups=baseus.groups, events=baseus.events, **baseus.params)
    sim.run(dotlog=True)
    print (get_results(sim))


['30%', '55%', '83%', '0.48%', '36%', '0.58%', '73']


In [59]:
if SAVE:
    results = get_results(sim)
    db['5'] = results

In [61]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '5.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/5.mp4' controls>Video Failed to Load</video>
    

In [60]:
if RUN:
    md(results_table(*db['5']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   30% | 
| HIT      | 55%      | 
| Total      | 83%      | 
| Fatalities | 0.48%      | 
| % > 70 | 36%      | 
| IFR | 0.58%      | 
| Days to Peak |   73 | 


We've attempted to tailor the event setup so that the infection curve resembles that of [Example 1](#1.-Equal). The theory is that the $R_0$ measured in the early stages of a pandemic should be reflective of actual contacts, not the theoretical contacts of the model.

This scenario is not perfectly substitutable with Example 1, however.

One measure we can use to compare scenarios is the total number of interactions. We find that under the Event and Gated approaches it takes *more* contacts to result in the same level of spread.

Avg Number of Contacts per Subject:

Example 1: 50.3

Example 2: 52.3 

Example 3: 53.4

Example 5: 65.5

Example 6: 75.5

*Average of first 100 days across 5 sims for each scenario.*

Other issues with substitutability may exist and are being explored. SIR models determine R0 in the idealized environment of Example 1 and so may not be suitable for customized environments such as this one.

#### 6. Gates ####
We can further improve the real world relevance of interactions by introducing [gates](concepts.ipynb#Gates). Subjects are not always freely able to interact with all other people in a population. Often their movement is restricted to within certain areas. Furthermore, other people's access into those areas is restricted.

Elderly people living in retirement homes or assisted living centers is an example. To simulate this, we will split `group4` into two separate groups. 

* `group4a`
    * population of 600 (2/3s of `group4`) 
    * IFR of 4.2%
    * move freely throughout the entire grid as previously
* `group4b` 
    * population of 300 (1/3rd of `group4`)
    * IFR of 7.8%
    * movement restricted to 6x6 box

We have also adjusted the IFR on the basis that `group4b` is likely older and also probably more frail than `group4a`. IFRs approximate those found [here](https://www.publichealthontario.ca/-/media/documents/ncov/epi/2020/06/covid19-epi-case-identification-age-only-template.pdf?la=en).

In addition, we will add an event specifically for the new group inside the gate.

```python
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2, group3, group4a, group4b]

church2 = Event(
    name='church2', xy=[2,3], start_tick=7, 
    groups=[4], capacity=5, recurring=7
)

events_gated = [
    school1, school2, school3, game1, game2, game3, game4, 
    concert1, concert2, concert3, concert4, 
    bar1, bar2, bar3, bar4, bar5, 
    church, church2,
]
```

Now, such elderly populations are entirely sealed of from the rest of the world. In fact, they are often visited by family members or friends. We can mimick this with the use of a Travel object.

In this sim, at least one person will enter into the `group4b` gate for a day only. And this will repeat every day of the sim.

```python
from rknot.events import Travel

visit = Travel(
    name='visit', xy=[1,1], start_tick=3, 
    groups=[1,2], capacity=1, duration=1, recurring=1
)
events.append(visit)

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [62]:
if IMPORT:
    from rknot import Sim, Chart
    from rknot.sims import baseus
    
    modulereload(baseus)
    assert 'susf' not in baseus.groups_gated[1]    
    assert all(g['mover'] == 'local' for g in baseus.groups)
    
if RUN:
    sim = Sim(groups=baseus.groups_gated, events=baseus.events_gated, **baseus.params)
    sim.run()
    print (get_results(sim))


['29%', '58%', '83%', '0.59%', '49%', '0.71%', '66']


In [63]:
if SAVE:
    results = get_results(sim)
    db['6'] = results

In [65]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '6.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/6.mp4' controls>Video Failed to Load</video>
    

In [64]:
if RUN:
    md(results_table(*db['6']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   29% | 
| HIT      | 58%      | 
| Total      | 83%      | 
| Fatalities | 0.59%      | 
| % > 70 | 49%      | 
| IFR | 0.71%      | 
| Days to Peak |   66 | 


Again, the gated structure is intended to mimic [Example 1](#1.-Equal), [Example 3](#3.-Social), and [Example 5](#5.-Events).

The use for Events and Gates will become clear when we explore the impact of [Policy Decisions](#Policy).

#### 7. Pre-Immunity with Events and Gates ####
Now we can see how pre-immunity might impact viral spread in a population with more heterogeneous interactions.

```python
group2['susf'] = .8
group3['susf'] = .65
groups = [group1, group2, group3, group4a, group4b]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [67]:
if IMPORT:
    from rknot import Sim, Chart
    from rknot.sims import baseus
    modulereload(baseus)

    assert 'susf' not in baseus.groups_gated[1]    
    assert all(g['mover'] == 'local' for g in baseus.groups)

    groups_gated = baseus.groups_gated
    groups_gated[1]['susf'] = .8
    groups_gated[2]['susf'] = .65

if RUN:   
    sim = Sim(groups=groups_gated, events=baseus.events_gated, **baseus.params)
    sim.run()
    print (get_results(sim))


['17%', '38%', '63%', '0.42%', '39%', '0.67%', '91']


In [68]:
if SAVE:
    results = get_results(sim)
    db['7'] = results

In [70]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)
    chart.animate.save(PATH + '7.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/7.mp4' controls>Video Failed to Load</video>
    

In [69]:
if RUN:
    md(results_table(*db['7']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   17% | 
| HIT      | 38%      | 
| Total      | 63%      | 
| Fatalities | 0.42%      | 
| % > 70 | 39%      | 
| IFR | 0.67%      | 
| Days to Peak |   91 | 


This is the most interesting curve to-date. We see that an outbreak in the 70+G group occurs very early, resulting a small early peak. This gives the impression of a second wave of infections, when in reality the virus was always spreading consistently in the main population.

Despite the early outbreak, once again we see that pre-existing immunity would have the effect of reducing peak infections and HIT.

Fatalities were also reduced despite the outbreak in the elderly population, thanks to a lower level of total spread.

#### 8. Self Aware Social Distancing ####
In a self-aware population, we can also incorporate an assumption that certain members of the population will implement social distancing practices even in the absence of prescribed government policy. For example, individuals might wear masks or face shields while in public. 

This is implemented via a SocialDistancing object, which reduces the transmission factor of the subjects in the applicable group.

```python
from rknot.events import SocialDistancing as SD

sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
events.append(sd)

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [74]:
if IMPORT:
    from rknot import Sim, Chart
    from rknot.sims import baseus
    modulereload(baseus)
    groups_gated = baseus.groups_gated
    events_gated = baseus.events_gated
    
    assert 'susf' not in baseus.groups_gated[1]
    assert len(events_gated) == 19

    from rknot.events import SocialDistancing as SD
    
    sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
    events_gated.append(sd)

if RUN:
    sim = Sim(groups=groups_gated, events=events_gated, **baseus.params)
    sim.run()
    print (get_results(sim))


['26%', '55%', '81%', '0.49%', '37%', '0.60%', '79']


In [75]:
if SAVE:
    results = get_results(sim)
    db['8'] = results

In [77]:
if SAVE_CHARTS:
    sim1
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + '8.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/8.mp4' controls>Video Failed to Load</video>
    

In [76]:
if RUN:
    md(results_table(*db['8']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   26% | 
| HIT      | 55%      | 
| Total      | 81%      | 
| Fatalities | 0.49%      | 
| % > 70 | 37%      | 
| IFR | 0.60%      | 
| Days to Peak |   79 | 


Social distancing does flatten the infection curve, resulting in a modest decrease in peak infections and HIT and delaying the peak slightly. Fatalities are also reduced.

#### 9. Self Aware Social Distancing + Pre-Immunity ####
In a self-aware population, we can also incorporate an assumption that certain members of the population will implement social distancing practices (even in the absence of prescribed government policy). For example, individuals might wear masks or face shields while in public. 

This is implemented via a SocialDistancing object, which reduces the transmission factor of the subjects in the applicable group.

```python
from rknot.events import SocialDistancing as SD

sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
events.append(sd)

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [82]:
if IMPORT:
    from rknot import Sim, Chart
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]

    from rknot.events import SocialDistancing as SD
    
    groups_gated = baseus.groups_gated
    events_gated = baseus.events_gated

    groups_gated[1]['susf'] = .8
    groups_gated[2]['susf'] = .65    


    sd = SD(name='6-feet', tmfs=[.975,.95,.75,.5], groups=[1,2,3,4], start_tick=5, duration=90)
    events_gated.append(sd)
if RUN:
    sim = Sim(groups=groups_gated, events=events_gated, **baseus.params)
    sim.run()
    print (get_results(sim))


['22%', '43%', '63%', '0.40%', '35%', '0.64%', '122']


In [85]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + '9.mp4', writer=FFMpegWriter(fps=10))

In [83]:
if SAVE:
    results = get_results(sim)
    db['9'] = results

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/9.mp4' controls>Video Failed to Load</video>
    

In [84]:
if RUN:
    md(results_table(*db['9']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   22% | 
| HIT      | 43%      | 
| Total      | 63%      | 
| Fatalities | 0.40%      | 
| % > 70 | 35%      | 
| IFR | 0.64%      | 
| Days to Peak |   122 | 


Here we see that just the combination of pre-immunity and a modest amount of social distancing reduces all aspects of the infection curve.

Social distancing among the most susceptible (i.e. `group4b`) was a key factor in a the reduced fatality rate.

Essentially, in this simulation, social distancing prevented an outbreak in the `group4b` community (whereas outbreaks had previously occured there in prior sims).

While these are all positives, note also that the outbreak occurs only AFTER social distancing is abandoned. So social distancing practices are helpful only so long as they are maintained.

#### 10. Isolation ####
TBD

### Policy ###

With a more realistic model of subject interaction, we can begin to experiment with the impact of different policy measures.

*RKnot* can simulate policy measures via `Restriction`, `SocialDistancing`, and `Quarantine` objects. Further details [here](concepts.ipynb#Restrictions).

The simulations are based on [this scenario](#6.-Gates) and so the impact of the policy measures contemplated should be considered relative to that scenario.

The structure can be imported as follows:

```python
from rknot.sims.baseus import params, events_gated, groups_gated
```

In [35]:
### HIDDEN ###
import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.animation import FFMpegWriter

from rknot import Sim, Chart

#### 1. Restrict Large Gatherings ####
We'll start by simply restricting large gathers, which for this sim is any event with 10+ capacity (0.1% of the entire population). The restrictions will last for 60 days.

When considering capacity, remember that a 100,000-seat stadium in a 10 million person catchment represents 1% of the population.

We assume this policy are implemented on day 30, when the population finally realizes there is a pandemic and government has had time to implement prevention measures.

The restriction will last for 60 days.

```python
from rknot.events import Restriction

large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
events_w_res = events_gated + [large_gatherings]

sim = Sim(groups=groups_gated, events=events_w_res, details=True, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [86]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]    
    
    from rknot.events import Restriction

    large_gatherings = Restriction(
        name='large', start_tick=30, duration=120, criteria={'capacity': 10}
    )
    events_w_res = baseus.events_gated + [large_gatherings]
if RUN:
    sim = Sim(groups=baseus.groups_gated, events=events_w_res, **baseus.params)
    sim.run()
    print (get_results(sim))


---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           Boundary|  [  1 100   1 100]|          Locations|             10,000|
|-------------------|-------------------|-------------------|-------------------|
|         Population|             10,000|            Density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|       Contact Rate|                1.0|                   |                   |
|-------------------|-------------------|-------------------|-------------------|



['15%', '37%', '72%', '0.44%', '34%', '0.61%', '86']


In [87]:
if SAVE:
    results = get_results(sim)
    db['large'] = results

In [89]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + 'large.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/large.mp4' controls>Video Failed to Load</video>

In [90]:
if RUN:
    md(results_table(*db['large']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   15% | 
| HIT      | 37%      | 
| Total      | 72%      | 
| Fatalities | 0.44%      | 
| % > 70 | 34%      | 
| IFR | 0.61%      | 
| Days to Peak |   86 | 


We see that the curve is flattened significantly during the restriction period. Infections have a much lower peak, BUT also a much longer tail (evident by the still high relatively high level of total infections).

This results as the event restriction is lifted, which leads to a slower rate of decline of the virus.

#### 2. Social Distancing ####
We can mimick the implementation of Social Distancing policies in certain settings via the `SocialDistancing` object. Mask wearing, hand sanitizing, and 6-foot perimeters all provide varying levels of protection.

It is hard to estimate the degree of protection from each, and even harder in combination. For instance, [this study](https://pubmed.ncbi.nlm.nih.gov/23498357/) found anywhere between a 1.1- and 55-fold reduction in exposure to influenza with varying mask designs.

So we provide `tmfs` here for illustration purposes only, attempting to catch all social distancing practices. We also provide different `tmfs` for the different age groups, meant to simulate adherence to policy.

The policy measure is implemented on day 30 and maintained for 120 days.

```python
from rknot.events import SocialDistancing as SD

sd = SD(
    name='all', tmfs=[.8,.8,.7,.65,.5], 
    groups=[0,1,2,3,4], start_tick=30, duration=120
)
events_w_res = events_gated + [sd]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [95]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]
if RUN:
    from rknot.events import SocialDistancing as SD

    sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
    events_w_res = baseus.events_gated + [sd]

    sim = Sim(groups=baseus.groups_gated, events=events_w_res, **baseus.params)
    sim.run()
    print (get_results(sim))

['10%', '21%', '68%', '0.38%', '28%', '0.56%', '73']


In [100]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + 'policy_sd.mp4', writer=FFMpegWriter(fps=10))

In [98]:
if SAVE:
    results = get_results(sim)
    db['policy_sd'] = results

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/policy_sd.mp4' controls>Video Failed to Load</video>
       

In [9]:
if RUN:
    md(results_table(*db['policy_sd']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   10% | 
| HIT      | 21%      | 
| Total      | 68%      | 
| Fatalities | 0.38%      | 
| % > 70 | 28%      | 
| IFR | 0.56%      | 
| Days to Peak |   73 | 


We can see that social distancing is certainly the most effective approach in terms of "flattening the curve" with the lowest HIT and peak infections seen thus far. 

In this sim, however, social distancing still allows for an early, significant outbreak in the 70+G group (even while the restrictions are in place). This create a seemingly rapid increase in infections similar to a standard curve. Once HIT is reached within the 70+G group, social distancing practices in the broader population are able to temper spread and the curve declines.

The virus continues to linger, however, and infections increase in smaller second wave once the social distancing policy is lifted.

#### 3. Restrict Elderly Visits ####
We can restrict events by name by passing `name` key to `criteria`. We can do this to restrict the travel event to the `70+G` area. 

The policy measure is implemented on day 30 and maintained for another 120 days. There will be no other restrictions.

```python
no_visits = Restriction(
    name='no_visits', start_tick=30, 
    duration=120, criteria={'name': 'visit'}
)
events_w_res = events_gated + [no_visits]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [91]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]
if RUN:
    from rknot.events import Restriction    
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
    )
    events_w_res = baseus.events_gated + [no_visits]

    sim = Sim(groups=baseus.groups_gated, events=events_w_res, **baseus.params)
    sim.run()
    print (get_results(sim))


['30%', '55%', '80%', '0.29%', '20%', '0.36%', '64']


In [92]:
if SAVE:
    results = get_results(sim)
    db['no_visits'] = results

In [94]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + 'no_visits.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/baseus/no_visits.mp4' controls>Video Failed to Load</video>
    

In [93]:
md(results_table(*db['no_visits']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   30% | 
| HIT      | 55%      | 
| Total      | 80%      | 
| Fatalities | 0.29%      | 
| % > 70 | 20%      | 
| IFR | 0.36%      | 
| Days to Peak |   64 | 


By simply quarantining the elderly, we see the most dramatic reduction in fatalities of any scenario thus far. An outbreak NEVER occurs in the 70+G gated area, even AFTER the policy restrictions are lifted, because the virus has already been eradicated by that time. 

We can see here that allowing a *high level of infection* among the **least susceptible** has resulted in a *low level of fatalities* among the **most susceptible**.

This is a controversial approach and does have difficult ethical implications, but its power to reduce death cannot be ignored.

#### 4. Social Distancing and Restrict Elderly Visits and Large Gatherings ####
What happens if we combine the three approaches above?

```python
large_gatherings = Restriction(
    name='large', start_tick=30, duration=60, criteria={'capacity': 10}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, 
    duration=120, criteria={'name': 'visit'}
)
sd = SD(
    name='all', tmfs=[.8,.8,.7,.65,.5], 
    groups=[0,1,2,3,4], start_tick=30, duration=120
)
events_w_res = events_gated + [large_gatherings, no_visits, sd]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [101]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]
if RUN:
    large_gatherings = Restriction(
        name='large', start_tick=30, duration=120, criteria={'capacity': 10}
    )
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
    )
    sd = SD(name='all', tmfs=[.8,.8,.7,.65,.5], groups=[0,1,2,3,4], start_tick=30, duration=120)
    events_w_res = baseus.events_gated + [large_gatherings, no_visits, sd]

    sim = Sim(groups=baseus.groups_gated, events=events_w_res, **baseus.params)
    sim.run()
    print (get_results(sim))


['24%', '55%', '81%', '0.53%', '39%', '0.65%', '205']


In [102]:
if SAVE:
    results = get_results(sim)
    db['sd_large_no_visits'] = results

In [104]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)    
    chart.animate.save(PATH + 'sd_&_no_large_&_no_visits.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
    src='https://storage.googleapis.com/rknotvids/baseus/sd_%26_no_large_%26_no_visits.mp4' controls>Video Failed to Load</video>
    

In [103]:
md(results_table(*db['sd_large_no_visits']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   24% | 
| HIT      | 55%      | 
| Total      | 81%      | 
| Fatalities | 0.53%      | 
| % > 70 | 39%      | 
| IFR | 0.65%      | 
| Days to Peak |   205 | 


Incredibly, the combination of restrictions leads to worse outcomes than any of the individual restrictions on their own.

While in place, the restrictions do tightly contain infections and signficantly delay the onset of the pandemic. But once they are lifted, the virus spreads unabated through a still highly susceptible population.

i.e. Not enough subjects have achieved immunity by the time the restrictions are lifted.

#### 5. Quarantine ####
We can even mimick the impact of quarantines via the Quarantine object.

Here we show the impact of a 30-day quarantine for all groups in the Sim.

```python
from rknot.events import Quarantine

quarantine = Quarantine(
    name='all', start_tick=30, 
    groups=[0,1,2,3,4], duration=30
)
events_w_res = events_gated + [quarantine]

sim = Sim(groups=groups_gated, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [109]:
if IMPORT:
    from rknot.sims import baseus
    modulereload(baseus)
    assert len(baseus.events_gated) == 19
    assert 'susf' not in baseus.groups_gated[1]
if RUN:
    from rknot.events import Quarantine

    quarantine = Quarantine(name='all', start_tick=30, groups=[0,1,2,3,4], duration=30)
    events_w_res = events_gated + [quarantine]

    sim = Sim(groups=groups_gated, events=events_w_res, **params)
    sim.run()
    print (get_results(sim))


['0%', '0%', '0%', '0.00%', '0%', '0.00%', '34']


In [112]:
if SAVE:
    results = get_results(sim)
    db['quarantine'] = results

In [113]:
if SAVE_CHARTS:
    chart = Chart(sim, use_init_func=True)        
    chart.animate.save(PATH + 'quarantine.mp4', writer=FFMpegWriter(fps=10))

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/quarantine.mp4' controls>Video Failed to Load</video>
       

In [111]:
md(results_table(*db['quarantine']))

Results: 

|      |      | 
| ------------- |:-------------:| 
| Peak      |   0% | 
| HIT      | 0%      | 
| Total      | 0%      | 
| Fatalities | 0.00%      | 
| % > 70 | 0%      | 
| IFR | 0.00%      | 
| Days to Peak |   34 | 


The pandemic never materializes in this simulation. The quarantine eradicates the virus swiftly. 

This is an interesting result. This obvioulsy did not occur in many places in the real world that took this approach. There are several possible reasons worth considering:

* Random Variation: if you run this simulation multiple times, you will note some sims where spread does occur.
* Scale: a larger simulation would increase the possibility that a very small number of subjects can continue to pass around the virus while it is muted in the broader population
* Adherence: real-world adherence to the implemented policies was much lower than this simulation suggests.
* Inconsistency: some areas implemented strict quarantines while others did not, and there was mixing among those populations.

### A Fulsome Scenario ###

We'll now create a more fulsome scenario encompassing many of the factors discussed.

Adjustments to prior scenarios include:

* add additional group with `mover='social'` in the 20-49 cohort. this cohort also flaunts social distancing guidelines as so its tmf in the SD object is `1.25`
* add susceptibility factors for most groups
* more fine-tuned capacity restrictions that gradually allow larger events over time
* restrict visits to the elderly for a full year

```python
### GROUPS ###
group1 = dict(
    name='0-19',
    n=2700,
    n_inf=0,
    ifr=0.00003,
    mover='local',
)
group2a = dict(
    name='20-49L',
    n=3900,
    n_inf=1,
    ifr=0.0002,
    mover='local',
    susf=0.85,    
)
group2b = dict(
    name='20-49S',
    n=200,
    n_inf=1,
    ifr=0.0002,
    mover='social',
    susf=0.85,    
)
group3 = dict(
    name='50-69',
    n=2300,
    n_inf=1,
    ifr=0.005,
    susf=.7,
    mover='local',
)
group4a = dict(
    name='70+',
    n=600,
    n_inf=0,
    ifr=0.042,
    mover='local',
    susf=0.9,    
)
group4b = dict(
    name='70+G',
    n=300,
    n_inf=0,
    ifr=0.0683,
    mover='local',
    box=[1,6,1,6],
    box_is_gated=True,
)
groups = [group1, group2a, group2b, group3, group4a, group4b]
params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

### EVENTS ###
from rknot.events import Event, Travel

school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1,2], capacity=50, recurring=14)                 
concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1,2], capacity=50, recurring=14)
bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1,2], capacity=5, recurring=7)
bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1,2], capacity=5, recurring=7)
bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2,3], capacity=5, recurring=7)
church = Event(name='church', xy=[2,91], start_tick=7, groups=[3,4], capacity=20, recurring=7)
church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[5], capacity=5, recurring=7)

visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2,3], capacity=1, duration=1, recurring=1)

events = [
    school1, school2,
    game, concert1, concert2, bar, bar2, bar3, 
    church, church2, visit,
]

### RESTRICTIONS ###
small_gatherings = Restriction(
    name='small', start_tick=30, duration=30, criteria={'capacity': 5}
)
med_gatherings = Restriction(
    name='med', start_tick=60, duration=30, criteria={'capacity': 10}
)
large_gatherings = Restriction(
    name='large', start_tick=90, duration=30, criteria={'capacity': 50}
)
no_visits = Restriction(
    name='no_visits', start_tick=30, duration=365, criteria={'name': 'visit'}
)
sd = SD(name='all', tmfs=[.8, .8, 1.25, .7,.65,.5], groups=[0,1,2,3,4,5], start_tick=30, duration=120)
events_w_res = events + [small_gatherings, med_gatherings, large_gatherings, no_visits, sd]

sim = Sim(groups=groups, events=events_w_res, **params)
sim.run()

chart = Chart(sim, use_init_func=True)
chart.animate.to_html5_video()
```

In [ ]:
### HIDDEN ###
if RUN:
    ### GROUPS ###
    group1 = dict(
        name='0-19',
        n=2700,
        n_inf=0,
        ifr=0.00003,
        mover='local',
    )
    group2a = dict(
        name='20-49L',
        n=3900,
        n_inf=1,
        ifr=0.0002,
        mover='local',
        susf=0.85,    
    )
    group2b = dict(
        name='20-49S',
        n=200,
        n_inf=1,
        ifr=0.0002,
        mover='social',
        susf=0.85,    
    )
    group3 = dict(
        name='50-69',
        n=2300,
        n_inf=1,
        ifr=0.005,
        susf=.7,
        mover='local',
    )
    group4a = dict(
        name='70+',
        n=600,
        n_inf=0,
        ifr=0.042,
        mover='local',
        susf=0.9,    
    )
    group4b = dict(
        name='70+G',
        n=300,
        n_inf=0,
        ifr=0.0683,
        mover='local',
        box=[1,6,1,6],
        box_is_gated=True,
    )
    groups = [group1, group2a, group2b, group3, group4a, group4b]
    params = {'dlevel': 'med', 'Ro':2.5, 'days': 365, 'imndur': 365, 'infdur': 14}

    ### EVENTS ###
    from rknot.events import Event, Travelx

    school1 = Event(name='school1', xy=[25,42], start_tick=2, groups=[0], capacity=10, recurring=2)
    school2 = Event(name='school2', xy=[78,82], start_tick=3, groups=[0], capacity=10, recurring=2)
    game = Event(name='game', xy=[50,84], start_tick=6, groups=[0,1,2,3], capacity=100, recurring=14)
    concert1 = Event(name='concert1', xy=[20,20], start_tick=7, groups=[1,2], capacity=50, recurring=14)                 
    concert2 = Event(name='concert2', xy=[91,92], start_tick=21, groups=[1,2], capacity=50, recurring=14)
    bar = Event(name='bar', xy=[17,24], start_tick=4, groups=[1,2], capacity=5, recurring=7)
    bar2 = Event(name='bar2', xy=[87,13], start_tick=5, groups=[1,2], capacity=5, recurring=7)
    bar3 = Event(name='bar3', xy=[52,89], start_tick=6, groups=[1,2,3], capacity=5, recurring=7)
    church = Event(name='church', xy=[2,91], start_tick=7, groups=[3,4], capacity=20, recurring=7)
    church2 = Event(name='church', xy=[2,3], start_tick=7, groups=[5], capacity=5, recurring=7)

    visit = Travel(name='visit', xy=[1,1], start_tick=3, groups=[1,2,3], capacity=1, duration=1, recurring=1)

    events = [
        school1, school2,
        game, concert1, concert2, bar, bar2, bar3, 
        church, church2, visit,
    ]

    ### RESTRICTIONS ###
    small_gatherings = Restriction(
        name='small', start_tick=30, duration=30, criteria={'capacity': 5}
    )
    med_gatherings = Restriction(
        name='med', start_tick=60, duration=30, criteria={'capacity': 10}
    )
    large_gatherings = Restriction(
        name='large', start_tick=90, duration=30, criteria={'capacity': 50}
    )
    no_visits = Restriction(
        name='no_visits', start_tick=30, duration=120, criteria={'name': 'visit'}
    )
    sd = SD(name='all', tmfs=[.8, .8, 1.25, .7,.65,.5], groups=[0,1,2,3,4,5], start_tick=30, duration=120)
    events_w_res = events + [small_gatherings, med_gatherings, large_gatherings, no_visits, sd]

    sim = Sim(groups=groups, events=events_w_res, **params)
    sim.run()

    chart = Chart(sim, use_init_func=True)
if SAVE_CHARTS:
    chart.animate.save(PATH + 'fulsome.mp4', writer=FFMpegWriter(fps=10))

In [ ]:
if SAVE:
    results = get_results(sim)
    db['ful'] = results

<video width='705' height='340'
       src='https://storage.googleapis.com/rknotvids/baseus/fulsome.mp4' controls>Video Failed to Load</video>

In [4]:
md(results_table(*db['ful']))

The virus barely survives for the first 6-months of the outbreak, then as per other scenarios, once restrictions are lifted an outbreak occurs. Still, when the outbreak does occur, it is characterized by one of the lowest peaks and HITs in our analysis (due in part to the pre-immmunity of some of the groups).

And it achieves the lowest fatality rate of the group, mainly by restricting access to the elderly population for the duration of the pandemic and ensuring an outbreak never occurs in that region.

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [6]:
move_to_doc_folder()